In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from snowflake.ml.feature_store import FeatureView, Entity, FeatureStore
from dotenv import load_dotenv
from phmlondon.snow_utils import SnowflakeConnection

In [4]:
load_dotenv()
conn = SnowflakeConnection()
conn.use_database("INTELLIGENCE_DEV")
conn.use_schema("AI_CENTRE_DEV")

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://login.microsoftonline.com/8076439c-5b1f-4e15-91ea-a0bff0b3bf16/saml2?SAMLRequest=nZJPc9owEMW%2Fikc925bMnxoNkCGhaUlJS8B0ptyEvQYNsuRKMg58%2BsoGZtJDcuhNI73d39O%2BHd69FsI7gjZcyREiAUYeyFRlXO5GaJ08%2BjHyjGUyY0JJGKETGHQ3HhpWiJJOKruXS%2FhTgbGeayQNbR5GqNKSKma4oZIVYKhN6WryPKdRgCkzBrR1OHQtyQx3rL21JQ3Duq6DuhMovQsjjHGIB6FTNZJP6A2i%2FJhRamVVqsSt5NX96R0ECXG3QTiFIyyuhfdcXkbwEWV7ERn6LUkW%2FuLnKkHe5Pa7ByVNVYBegT7yFNbL%2BcWAcQ5WX3p9EveD6uAb5WYYsHOlITBS1blgB0hVUVbWtQ7cKcwhC4XacTew2XSEygPPZvFmCvf5%2FIk%2F7dMXlqxhsFx%2FX%2B00PzxvWHH%2Bmh3PETnFv3cvKfJ%2B3eKNmnhnxlQwk02o1l3hqOfjyI%2B6SYQpiWkvDqJOZ4O8qQuVS2bbypvz1kdQ8FQro3KrpOASWpcx%2Ftzvdgap39uS3O8C6fkDAsxneJvneNvZ5qQfNtFF6LI%2BtDWix%2F81lGH4tsV1HX%2B4hGbThRI8PXmPShfMvh8gCUh7wzM%2Fb6UUCsbFJMs0GOOCFELVDxqYdV

In [5]:
counts_by_patient = conn.session.sql("""
select o.patient_id, o.core_concept_id, count(o.core_concept_id) as code_count, current_timestamp() as last_updated
from prod_dwh.analyst_primary_care.observation as o
group by o.patient_id, o.core_concept_id
limit 1000;
""")

In [6]:
print(counts_by_patient)

In [7]:
fs = FeatureStore(
    session=conn.session,
    database="INTELLIGENCE_DEV",
    name="TEST_FEATURE_STORE_IW",
    default_warehouse="INTELLIGENCE_XS"
)

In [8]:
fs.list_entities().show()

----------------------------------------------------------------------------------------
|"NAME"        |"JOIN_KEYS"          |"DESC"                 |"OWNER"                  |
----------------------------------------------------------------------------------------
|CODES         |["CORE_CONCEPT_ID"]  |snomed clinical codes  |UG_INTELLIGENCE_DEVELOP  |
|DOLOCATIONID  |["DOLOCATIONID"]     |Drop off location id.  |UG_INTELLIGENCE_DEVELOP  |
|PATIENTS      |["PATIENT_ID"]       |patients               |UG_INTELLIGENCE_DEVELOP  |
|TRIP_ID       |["TRIP_ID"]          |Trip id.               |UG_INTELLIGENCE_DEVELOP  |
----------------------------------------------------------------------------------------



In [9]:
entity_patients = Entity(name="patients", join_keys=["patient_id"], desc="patients")
entity_codes = Entity(name="codes", join_keys=["core_concept_id"], desc="snomed clinical codes")
print(entity_patients)
print(entity_codes)

Entity(name=PATIENTS, join_keys=['PATIENT_ID'], owner=None, desc=patients)
Entity(name=CODES, join_keys=['CORE_CONCEPT_ID'], owner=None, desc=snomed clinical codes)


In [25]:
fs.register_entity(entity_patients)
fs.register_entity(entity_codes)

Entity(name=CODES, join_keys=['CORE_CONCEPT_ID'], owner=UG_INTELLIGENCE_DEVELOP, desc=snomed clinical codes)

In [10]:
fs.list_entities().show()

----------------------------------------------------------------------------------------
|"NAME"        |"JOIN_KEYS"          |"DESC"                 |"OWNER"                  |
----------------------------------------------------------------------------------------
|CODES         |["CORE_CONCEPT_ID"]  |snomed clinical codes  |UG_INTELLIGENCE_DEVELOP  |
|DOLOCATIONID  |["DOLOCATIONID"]     |Drop off location id.  |UG_INTELLIGENCE_DEVELOP  |
|PATIENTS      |["PATIENT_ID"]       |patients               |UG_INTELLIGENCE_DEVELOP  |
|TRIP_ID       |["TRIP_ID"]          |Trip id.               |UG_INTELLIGENCE_DEVELOP  |
----------------------------------------------------------------------------------------



In [11]:
fv = FeatureView(
    name="counts_by_patient",
    entities=[entity_patients, entity_codes],
    feature_df=counts_by_patient,
    timestamp_col="last_updated",
    refresh_freq="7 days",
    desc="Counts of a particular code by patient"   
)

In [30]:
registered_fv: FeatureView = fs.register_feature_view(
    feature_view = fv,
    version = "1",
    block = True,
    overwrite = False,
)

/Users/isobelweinberg/Documents/code/onelondon_snowflake_datascience/.venv/lib/python3.11/site-packages/snowflake/ml/feature_store/feature_store.py:1877: UserWarning: Your pipeline won't be incrementally refreshed due to: "Change tracking is not supported on queries with LIMIT.".
  self._check_dynamic_table_refresh_mode(feature_view_name)
